In [2]:
import requests

In [108]:
def get_seq_length():
    return 125

In [109]:
args = {
    "max_sequence_length": get_seq_length(),
    "epochs": 100,
    "learning_rate": 0.0001,
    "batch_size": 15
}
requests.post("http://localhost:5000/api/model", json=args)

<Response [200]>

In [27]:
def get_whois():
    return requests.get("https://hacker-news.firebaseio.com/v0/user/whoishiring/submitted.json").json()
    

In [28]:
def get_post(id):
    return requests.get(f"https://hacker-news.firebaseio.com/v0/item/{id}.json").json()

In [29]:
get_post(23379196)['title']

'Ask HN: Who is hiring? (June 2020)'

In [30]:
def is_hiring_post(post):
    return post.get("title", "").startswith("Ask HN: Who")

In [31]:
resp = get_post(23379196)

In [33]:
is_hiring_post(resp)

True

In [32]:
job_posts = resp["kids"]

In [82]:
def get_jobs(ids):
    jobs = []
    for id in ids:
        resp = get_post(id)
        if "text" in resp:
            jobs.append({"id": id, "text": resp["text"]})

    return jobs

In [36]:
jobs = get_jobs(job_posts)

In [37]:
jobs = [ job for job in jobs if len(job['text']) ]

In [95]:
def get_prediction(job):
    resp = requests.post("http://localhost:5000/api/model/predict", json={ 'max_sequence_length': get_seq_length(), 'text': job['text'] })
    return resp.json()

In [97]:
predictions = []
for job in jobs[0:10]:
    result = get_prediction(job)
    result['id'] = job['id']
    predictions.append(result)
    if 'preferred' in result and result['preferred'] == 1:
        print(f"Looks like you're interested in {job['id']}")

Looks like you're interested in 23380030
Looks like you're interested in 23387293
Looks like you're interested in 23379860
Looks like you're interested in 23384303
Looks like you're interested in 23385194


In [45]:
[x["id"] for x in jobs[0:5]]

[23380030, 23387293, 23449132, 23385642, 23379232]

In [101]:
[x for x in jobs if x['id'] == 23385194 ][0]['text']

'Twenty | Non-Profit | Frontend, Fullstack | San Francisco, Remote (US only) | $135k<p>Hi, we’re Twenty! We’re a brand new non-profit on a mission to discover and develop the potential in everyone. We are pre-product and pre-website, but we already have an incredible group of supporters. Sal Khan, founder and CEO of Khan Academy and Bill Jackson, founder of GreatSchools, are both on our board. The Eric and Wendy Schmidt Strategic Innovation Fund is our first funder and Schmidt Futures is our first customer. We&#x27;re launching our MVP (Hello World) later this month.<p>Our CEO is an education executive with 30+ years of experience. Our CPO is a former startup founder and product manager from Khan Academy, and our CTO sold his previous company to Google.<p>Our tech stack uses Typescript + React Native on the frontend with a light Rails backend running on App Engine. We’re still early and nothing is set in stone, so there’s lots of room to craft and improve our stack.<p>We&#x27;re lookin

In [73]:
def get_text(id):
    return [ x['text'] for x in jobs if  x['id'] == id ][0]['text']

In [74]:
get_text(23382696)

TypeError: string indices must be integers

In [84]:
len(predictions)

10

In [77]:
len(job_posts)

498